In [76]:
import pandas as pd
from numpy import nan
import numpy as np
from os import listdir
from os.path import isfile, join
from itertools import chain
def cleaner(tabl):
    tabl['Задание ГОЗ'] = tabl['Задание ГОЗ'].fillna(method='ffill')
    tabl['Наименование закупки'] = tabl['Наименование закупки'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].fillna(method='ffill')
    tabl['КБК'] = tabl['КБК'].astype('object')
    tabl = tabl.drop(index=[0,1])
    return tabl
def get_tovd_stats(tovds):
    def wrapper(row):
        tovd_stats = []
        for tovd in tovds:
            if row[tovd] != nan:
                tovd_stats.append([tovd, row["КБК"], row["Задание ГОЗ"], row["Наименование закупки"], row[tovd], row["Номер письма"]])
        return tovd_stats
    return wrapper

In [77]:
#Переменная для обновления списка (чтобы не задвоилось/троилось)
all_files = []
path = 'Original'
all_name_files = [f for f in listdir(path) if isfile(join(path, f)) and (f.endswith(".xls") or f.endswith(".xlsx"))]

for name in all_name_files:
    source = pd.read_excel(path+"/"+name, header=1)
    del source[source.columns[-1]]
    source = cleaner(source)
    source ['Номер письма'] = name
    all_files.append(source)

In [78]:
new_all_files=[]

for file in all_files:
    t=file
    tovd=t.columns[3:-1]
    t=t.apply(get_tovd_stats(tovd), axis=1)
    t = list(chain.from_iterable(t))
    new_all_files.append(pd.DataFrame(np.array(t),
                                      columns=['Бюджетополучатель', 'КБК', 
                                               'Задание ГОЗ', 'Наименование закупки', 
                                               'Сумма (руб)', '№ письма']))


,Задание ГОЗ,Наименование закупки,КБК,УМВД России по Хабаровскому краю,Номер письма
2,"Средства связи, организационной и вычислительн...",24 накопителя информации для оснащения серверн...,0302 0830292019 211,2095200.0,БН2.xlsx


In [79]:
new_all_files[0]['Сумма (руб)'] = pd.to_numeric(new_all_files[0]['Сумма (руб)'], errors='coerce')
print(new_all_files[0]['Сумма (руб)'].sum())

6046070111.219999
